In [13]:
from keras.models import Sequential
from keras.layers import *
from keras import layers
import numpy as np
from six.moves import range

# Parameters Config

In [14]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [15]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+ '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 64
LAYERS = 1

In [16]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [17]:
ctable = CharacterTable(chars)

In [18]:
ctable.indices_char

{0: ' ',
 1: '+',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

# Data Generation

In [19]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [20]:
print(questions[:5], expected[:5])

['380+76 ', '4+65   ', '36+195 ', '6+29   ', '23+951 '] ['456 ', '69  ', '231 ', '35  ', '974 ']


# Processing

In [21]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [22]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 12)
(18000, 4, 12)
Validation Data:
(2000, 7, 12)
(2000, 4, 12)
Testing Data:
(60000, 7, 12)
(60000, 4, 12)


In [23]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False  True False False False False False
   False]
  [False False False False False False False False False False False
    True]
  [False False False False  True False False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False False False False False
    True]
  [False False False False  True False False False False False False
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False False False False False False False False  True
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False False False False  True
   False]
  [False False False False  True False False False False False False
   False]
  [False False False False  True False False False False False False
   False]
  [ True False False False False False Fal

# Build Model

In [24]:
print('Build model...')

############################################
##### Build your own model here ############
############################################

model = Sequential()
model.add(LSTM(units=256, input_shape=(7, 12), activation='tanh'))
model.add(RepeatVector(4))
model.add(LSTM(units=128, activation='tanh', return_sequences=True))
model.add(Dense(units=128, activation='sigmoid'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=12, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam')

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 256)               275456    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 4, 256)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 4, 128)            197120    
_________________________________________________________________
dense_4 (Dense)              (None, 4, 128)            16512     
_________________________________________________________________
dense_5 (Dense)              (None, 4, 64)             8256      
_________________________________________________________________
dense_6 (Dense)              (None, 4, 12)             780       
Total params: 498,124
Trainable params: 498,124
Non-trainable params: 0
_______________________________________________________

# Training

In [25]:
for iteration in range(30):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 64s 4ms/step - loss: 1.9504 - val_loss: 1.8667
Q 111+250 T 361  ☒ 122 
Q 380+791 T 1171 ☒ 1111
Q 83+581  T 664  ☒ 111 
Q 486+165 T 651  ☒ 111 
Q 494+364 T 858  ☒ 1111
Q 31+515  T 546  ☒ 126 
Q 94+495  T 589  ☒ 111 
Q 701+672 T 1373 ☒ 1111
Q 44+683  T 727  ☒ 116 
Q 762+48  T 810  ☒ 111 

--------------------------------------------------
Iteration 1
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 31s 2ms/step - loss: 1.8259 - val_loss: 1.7584
Q 512+542 T 1054 ☒ 144 
Q 817+58  T 875  ☒ 144 
Q 892+264 T 1156 ☒ 1155
Q 16+311  T 327  ☒ 444 
Q 152+773 T 925  ☒ 144 
Q 246+418 T 664  ☒ 744 
Q 390+69  T 459  ☒ 744 
Q 46+534  T 580  ☒ 744 
Q 68+493

18000/18000 [==============================] - 9s 522us/step - loss: 0.7227 - val_loss: 0.7100
Q 59+938  T 997  ☒ 990 
Q 38+372  T 410  ☒ 401 
Q 97+789  T 886  ☒ 885 
Q 77+430  T 507  ☒ 509 
Q 67+150  T 217  ☒ 219 
Q 225+35  T 260  ☒ 259 
Q 412+179 T 591  ☑ 591 
Q 38+493  T 531  ☑ 531 
Q 865+1   T 866  ☒ 867 
Q 297+114 T 411  ☑ 411 

--------------------------------------------------
Iteration 15
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 9s 515us/step - loss: 0.6574 - val_loss: 0.5700
Q 379+516 T 895  ☒ 897 
Q 233+512 T 745  ☒ 746 
Q 785+341 T 1126 ☒ 1128
Q 34+363  T 397  ☒ 398 
Q 961+60  T 1021 ☒ 1023
Q 422+947 T 1369 ☒ 1378
Q 701+489 T 1190 ☒ 1199
Q 751+535 T 1286 ☒ 1287
Q 783+596 T 1379 ☒ 1380
Q 967+598 T 1565 ☒ 1573

--------------------------------------------------
Iteration 16
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 9s 513us/step - loss: 0.4951 - val_

18000/18000 [==============================] - 9s 517us/step - loss: 0.0145 - val_loss: 0.0231
Q 586+972 T 1558 ☑ 1558
Q 632+329 T 961  ☑ 961 
Q 416+945 T 1361 ☑ 1361
Q 200+24  T 224  ☑ 224 
Q 469+266 T 735  ☑ 735 
Q 380+791 T 1171 ☑ 1171
Q 52+587  T 639  ☑ 639 
Q 448+47  T 495  ☑ 495 
Q 43+887  T 930  ☑ 930 
Q 8+993   T 1001 ☑ 1001


# Testing

In [26]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
## ex. test_x = ["555+175", "860+7  ", "340+29 "]
## ex. test_y = ["730 ", "867 ", "369 "] 
#####################################################
    

MSG : Prediction


In [27]:
test_score = model.evaluate(test_x, test_y)

print("test score :", test_score)

60000/60000 [==============================] - 20s 335us/step
test score : 0.022868372689560056


In [28]:
output = model.predict_classes(test_x)

In [29]:
count = 0
for i in range(len(test_y[:10000])):
    q = ctable.decode(test_x[i])
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(output[i], calc_argmax=False)
    print(i, 'Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        count += 1
        print(colors.ok + '☑' + colors.close, end=' ')
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)

print("Accuracy : ", count/len(test_y[:10000]))

0 Q 368+597 T 965  ☑ 965 
1 Q 686+480 T 1166 ☑ 1166
2 Q 393+76  T 469  ☑ 469 
3 Q 37+729  T 766  ☑ 766 
4 Q 35+740  T 775  ☑ 775 
5 Q 473+435 T 908  ☑ 908 
6 Q 38+733  T 771  ☑ 771 
7 Q 94+783  T 877  ☑ 877 
8 Q 488+18  T 506  ☑ 506 
9 Q 30+164  T 194  ☑ 194 
10 Q 76+64   T 140  ☑ 140 
11 Q 71+290  T 361  ☑ 361 
12 Q 454+717 T 1171 ☑ 1171
13 Q 149+593 T 742  ☑ 742 
14 Q 373+936 T 1309 ☑ 1309
15 Q 329+224 T 553  ☑ 553 
16 Q 42+844  T 886  ☑ 886 
17 Q 592+70  T 662  ☑ 662 
18 Q 487+170 T 657  ☑ 657 
19 Q 20+902  T 922  ☑ 922 
20 Q 51+327  T 378  ☑ 378 
21 Q 38+170  T 208  ☑ 208 
22 Q 958+49  T 1007 ☒ 1097
23 Q 838+237 T 1075 ☑ 1075
24 Q 884+58  T 942  ☑ 942 
25 Q 138+74  T 212  ☑ 212 
26 Q 850+73  T 923  ☑ 923 
27 Q 633+97  T 730  ☑ 730 
28 Q 515+24  T 539  ☑ 539 
29 Q 945+484 T 1429 ☑ 1429
30 Q 427+381 T 808  ☑ 808 
31 Q 17+19   T 36   ☑ 36  
32 Q 674+64  T 738  ☑ 738 
33 Q 355+754 T 1109 ☑ 1109
34 Q 589+359 T 948  ☑ 948 
35 Q 88+350  T 438  ☑ 438 
36 Q 316+12  T 328  ☑ 328 
37 Q 53+172

330 Q 960+56  T 1016 ☑ 1016
331 Q 31+18   T 49   ☑ 49  
332 Q 13+776  T 789  ☑ 789 
333 Q 71+715  T 786  ☑ 786 
334 Q 84+40   T 124  ☑ 124 
335 Q 589+55  T 644  ☑ 644 
336 Q 61+670  T 731  ☑ 731 
337 Q 3+978   T 981  ☑ 981 
338 Q 578+72  T 650  ☑ 650 
339 Q 92+669  T 761  ☑ 761 
340 Q 710+503 T 1213 ☑ 1213
341 Q 13+962  T 975  ☑ 975 
342 Q 463+46  T 509  ☑ 509 
343 Q 639+860 T 1499 ☑ 1499
344 Q 77+145  T 222  ☑ 222 
345 Q 811+354 T 1165 ☑ 1165
346 Q 790+233 T 1023 ☑ 1023
347 Q 8+442   T 450  ☑ 450 
348 Q 67+618  T 685  ☑ 685 
349 Q 141+86  T 227  ☑ 227 
350 Q 82+144  T 226  ☑ 226 
351 Q 647+118 T 765  ☑ 765 
352 Q 630+1   T 631  ☑ 631 
353 Q 286+353 T 639  ☑ 639 
354 Q 347+813 T 1160 ☑ 1160
355 Q 953+74  T 1027 ☑ 1027
356 Q 944+101 T 1045 ☑ 1045
357 Q 64+33   T 97   ☑ 97  
358 Q 554+3   T 557  ☑ 557 
359 Q 56+776  T 832  ☑ 832 
360 Q 479+0   T 479  ☑ 479 
361 Q 8+594   T 602  ☑ 602 
362 Q 17+296  T 313  ☑ 313 
363 Q 950+332 T 1282 ☑ 1282
364 Q 810+28  T 838  ☑ 838 
365 Q 730+615 T 1345

615 Q 920+113 T 1033 ☑ 1033
616 Q 857+526 T 1383 ☑ 1383
617 Q 155+362 T 517  ☑ 517 
618 Q 95+94   T 189  ☑ 189 
619 Q 662+760 T 1422 ☑ 1422
620 Q 26+202  T 228  ☑ 228 
621 Q 217+465 T 682  ☑ 682 
622 Q 544+87  T 631  ☑ 631 
623 Q 19+437  T 456  ☑ 456 
624 Q 26+595  T 621  ☑ 621 
625 Q 985+0   T 985  ☑ 985 
626 Q 2+281   T 283  ☑ 283 
627 Q 589+94  T 683  ☑ 683 
628 Q 209+545 T 754  ☑ 754 
629 Q 526+437 T 963  ☑ 963 
630 Q 843+53  T 896  ☑ 896 
631 Q 797+18  T 815  ☑ 815 
632 Q 867+67  T 934  ☑ 934 
633 Q 897+291 T 1188 ☑ 1188
634 Q 721+85  T 806  ☑ 806 
635 Q 16+16   T 32   ☑ 32  
636 Q 586+35  T 621  ☑ 621 
637 Q 255+580 T 835  ☑ 835 
638 Q 506+752 T 1258 ☑ 1258
639 Q 649+38  T 687  ☑ 687 
640 Q 537+55  T 592  ☑ 592 
641 Q 989+71  T 1060 ☑ 1060
642 Q 40+335  T 375  ☑ 375 
643 Q 13+588  T 601  ☑ 601 
644 Q 7+771   T 778  ☑ 778 
645 Q 0+942   T 942  ☑ 942 
646 Q 70+673  T 743  ☑ 743 
647 Q 9+373   T 382  ☑ 382 
648 Q 102+552 T 654  ☑ 654 
649 Q 427+71  T 498  ☑ 498 
650 Q 353+927 T 1280

972 Q 20+247  T 267  ☑ 267 
973 Q 351+110 T 461  ☑ 461 
974 Q 68+337  T 405  ☑ 405 
975 Q 251+810 T 1061 ☑ 1061
976 Q 82+190  T 272  ☑ 272 
977 Q 472+961 T 1433 ☑ 1433
978 Q 54+906  T 960  ☑ 960 
979 Q 555+366 T 921  ☑ 921 
980 Q 108+839 T 947  ☑ 947 
981 Q 493+384 T 877  ☑ 877 
982 Q 186+692 T 878  ☑ 878 
983 Q 30+523  T 553  ☑ 553 
984 Q 24+892  T 916  ☑ 916 
985 Q 8+775   T 783  ☑ 783 
986 Q 879+68  T 947  ☑ 947 
987 Q 72+682  T 754  ☑ 754 
988 Q 918+68  T 986  ☑ 986 
989 Q 460+24  T 484  ☑ 484 
990 Q 273+30  T 303  ☑ 303 
991 Q 450+972 T 1422 ☑ 1422
992 Q 724+32  T 756  ☑ 756 
993 Q 596+97  T 693  ☑ 693 
994 Q 357+805 T 1162 ☑ 1162
995 Q 61+97   T 158  ☑ 158 
996 Q 7+136   T 143  ☑ 143 
997 Q 89+174  T 263  ☑ 263 
998 Q 839+6   T 845  ☑ 845 
999 Q 800+786 T 1586 ☑ 1586
1000 Q 633+81  T 714  ☑ 714 
1001 Q 547+808 T 1355 ☑ 1355
1002 Q 622+7   T 629  ☑ 629 
1003 Q 745+541 T 1286 ☑ 1286
1004 Q 496+73  T 569  ☑ 569 
1005 Q 937+736 T 1673 ☑ 1673
1006 Q 281+4   T 285  ☑ 285 
1007 Q 888+81

1330 Q 971+854 T 1825 ☑ 1825
1331 Q 432+748 T 1180 ☑ 1180
1332 Q 700+55  T 755  ☑ 755 
1333 Q 82+837  T 919  ☑ 919 
1334 Q 6+192   T 198  ☑ 198 
1335 Q 476+55  T 531  ☑ 531 
1336 Q 23+668  T 691  ☑ 691 
1337 Q 42+877  T 919  ☑ 919 
1338 Q 602+917 T 1519 ☑ 1519
1339 Q 397+562 T 959  ☒ 969 
1340 Q 626+678 T 1304 ☑ 1304
1341 Q 71+540  T 611  ☑ 611 
1342 Q 66+747  T 813  ☑ 813 
1343 Q 924+920 T 1844 ☑ 1844
1344 Q 860+78  T 938  ☑ 938 
1345 Q 559+659 T 1218 ☑ 1218
1346 Q 758+235 T 993  ☑ 993 
1347 Q 94+723  T 817  ☑ 817 
1348 Q 261+304 T 565  ☑ 565 
1349 Q 32+991  T 1023 ☑ 1023
1350 Q 35+63   T 98   ☑ 98  
1351 Q 573+75  T 648  ☑ 648 
1352 Q 547+990 T 1537 ☑ 1537
1353 Q 11+282  T 293  ☑ 293 
1354 Q 649+3   T 652  ☑ 652 
1355 Q 561+64  T 625  ☑ 625 
1356 Q 90+695  T 785  ☑ 785 
1357 Q 679+283 T 962  ☑ 962 
1358 Q 76+487  T 563  ☑ 563 
1359 Q 695+97  T 792  ☑ 792 
1360 Q 294+859 T 1153 ☑ 1153
1361 Q 8+179   T 187  ☑ 187 
1362 Q 42+886  T 928  ☑ 928 
1363 Q 956+263 T 1219 ☑ 1219
1364 Q 663+384

...

8459 Q 126+415 T 541  ☑ 541 
8460 Q 942+909 T 1851 ☑ 1851
8461 Q 53+690  T 743  ☑ 743 
8462 Q 972+95  T 1067 ☑ 1067
8463 Q 931+87  T 1018 ☑ 1018
8464 Q 554+5   T 559  ☑ 559 
8465 Q 74+1    T 75   ☑ 75  
8466 Q 410+48  T 458  ☑ 458 
8467 Q 403+459 T 862  ☑ 862 
8468 Q 55+437  T 492  ☑ 492 
8469 Q 48+596  T 644  ☑ 644 
8470 Q 396+989 T 1385 ☑ 1385
8471 Q 240+864 T 1104 ☑ 1104
8472 Q 308+65  T 373  ☑ 373 
8473 Q 30+689  T 719  ☑ 719 
8474 Q 94+66   T 160  ☑ 160 
8475 Q 16+663  T 679  ☑ 679 
8476 Q 109+403 T 512  ☑ 512 
8477 Q 14+713  T 727  ☑ 727 
8478 Q 989+91  T 1080 ☑ 1080
8479 Q 80+93   T 173  ☑ 173 
8480 Q 215+989 T 1204 ☑ 1204
8481 Q 21+583  T 604  ☑ 604 
8482 Q 3+581   T 584  ☑ 584 
8483 Q 925+682 T 1607 ☑ 1607
8484 Q 24+718  T 742  ☑ 742 
8485 Q 51+95   T 146  ☑ 146 
8486 Q 59+311  T 370  ☑ 370 
8487 Q 7+940   T 947  ☑ 947 
8488 Q 88+392  T 480  ☑ 480 
8489 Q 849+0   T 849  ☑ 849 
8490 Q 861+230 T 1091 ☑ 1091
8491 Q 516+289 T 805  ☑ 805 
8492 Q 12+631  T 643  ☑ 643 
8493 Q 450+623

8804 Q 914+231 T 1145 ☑ 1145
8805 Q 220+969 T 1189 ☒ 1188
8806 Q 97+166  T 263  ☑ 263 
8807 Q 11+750  T 761  ☑ 761 
8808 Q 375+809 T 1184 ☑ 1184
8809 Q 244+247 T 491  ☑ 491 
8810 Q 649+738 T 1387 ☑ 1387
8811 Q 522+369 T 891  ☑ 891 
8812 Q 418+65  T 483  ☑ 483 
8813 Q 46+684  T 730  ☑ 730 
8814 Q 99+369  T 468  ☑ 468 
8815 Q 999+464 T 1463 ☑ 1463
8816 Q 945+36  T 981  ☑ 981 
8817 Q 662+88  T 750  ☑ 750 
8818 Q 838+19  T 857  ☑ 857 
8819 Q 116+97  T 213  ☑ 213 
8820 Q 838+94  T 932  ☑ 932 
8821 Q 284+515 T 799  ☒ 709 
8822 Q 2+160   T 162  ☑ 162 
8823 Q 392+46  T 438  ☑ 438 
8824 Q 591+59  T 650  ☑ 650 
8825 Q 650+114 T 764  ☑ 764 
8826 Q 891+28  T 919  ☑ 919 
8827 Q 312+960 T 1272 ☑ 1272
8828 Q 70+389  T 459  ☑ 459 
8829 Q 924+89  T 1013 ☑ 1013
8830 Q 946+782 T 1728 ☑ 1728
8831 Q 38+328  T 366  ☑ 366 
8832 Q 15+307  T 322  ☑ 322 
8833 Q 240+1   T 241  ☑ 241 
8834 Q 266+49  T 315  ☑ 315 
8835 Q 39+403  T 442  ☑ 442 
8836 Q 12+322  T 334  ☑ 334 
8837 Q 461+83  T 544  ☑ 544 
8838 Q 81+834 

9141 Q 48+276  T 324  ☑ 324 
9142 Q 147+795 T 942  ☑ 942 
9143 Q 476+709 T 1185 ☑ 1185
9144 Q 684+493 T 1177 ☑ 1177
9145 Q 505+320 T 825  ☑ 825 
9146 Q 514+956 T 1470 ☑ 1470
9147 Q 84+822  T 906  ☑ 906 
9148 Q 157+688 T 845  ☑ 845 
9149 Q 101+975 T 1076 ☑ 1076
9150 Q 76+781  T 857  ☑ 857 
9151 Q 235+12  T 247  ☑ 247 
9152 Q 708+172 T 880  ☑ 880 
9153 Q 106+75  T 181  ☑ 181 
9154 Q 25+41   T 66   ☑ 66  
9155 Q 639+237 T 876  ☑ 876 
9156 Q 305+78  T 383  ☑ 383 
9157 Q 547+931 T 1478 ☑ 1478
9158 Q 957+204 T 1161 ☑ 1161
9159 Q 695+25  T 720  ☑ 720 
9160 Q 869+39  T 908  ☑ 908 
9161 Q 89+192  T 281  ☑ 281 
9162 Q 5+987   T 992  ☑ 992 
9163 Q 680+58  T 738  ☑ 738 
9164 Q 106+26  T 132  ☑ 132 
9165 Q 935+93  T 1028 ☑ 1028
9166 Q 370+74  T 444  ☑ 444 
9167 Q 933+914 T 1847 ☑ 1847
9168 Q 414+2   T 416  ☑ 416 
9169 Q 671+268 T 939  ☑ 939 
9170 Q 490+767 T 1257 ☑ 1257
9171 Q 701+36  T 737  ☑ 737 
9172 Q 91+818  T 909  ☑ 909 
9173 Q 521+77  T 598  ☑ 598 
9174 Q 227+513 T 740  ☑ 740 
9175 Q 576+92 

9509 Q 227+860 T 1087 ☑ 1087
9510 Q 57+40   T 97   ☑ 97  
9511 Q 610+113 T 723  ☑ 723 
9512 Q 825+95  T 920  ☑ 920 
9513 Q 681+18  T 699  ☑ 699 
9514 Q 821+905 T 1726 ☑ 1726
9515 Q 23+83   T 106  ☑ 106 
9516 Q 820+10  T 830  ☑ 830 
9517 Q 701+297 T 998  ☑ 998 
9518 Q 511+57  T 568  ☑ 568 
9519 Q 71+25   T 96   ☑ 96  
9520 Q 659+2   T 661  ☑ 661 
9521 Q 54+27   T 81   ☑ 81  
9522 Q 67+311  T 378  ☑ 378 
9523 Q 72+102  T 174  ☑ 174 
9524 Q 639+41  T 680  ☑ 680 
9525 Q 821+272 T 1093 ☑ 1093
9526 Q 615+741 T 1356 ☑ 1356
9527 Q 405+396 T 801  ☑ 801 
9528 Q 815+758 T 1573 ☑ 1573
9529 Q 167+57  T 224  ☑ 224 
9530 Q 204+140 T 344  ☑ 344 
9531 Q 596+81  T 677  ☑ 677 
9532 Q 384+90  T 474  ☑ 474 
9533 Q 464+30  T 494  ☑ 494 
9534 Q 42+758  T 800  ☑ 800 
9535 Q 19+566  T 585  ☑ 585 
9536 Q 985+525 T 1510 ☑ 1510
9537 Q 496+3   T 499  ☑ 499 
9538 Q 111+11  T 122  ☑ 122 
9539 Q 388+896 T 1284 ☑ 1284
9540 Q 976+41  T 1017 ☑ 1017
9541 Q 81+360  T 441  ☑ 441 
9542 Q 43+318  T 361  ☑ 361 
9543 Q 286+92 

9854 Q 66+834  T 900  ☒ 800 
9855 Q 755+732 T 1487 ☑ 1487
9856 Q 968+216 T 1184 ☑ 1184
9857 Q 500+85  T 585  ☑ 585 
9858 Q 642+897 T 1539 ☑ 1539
9859 Q 573+574 T 1147 ☑ 1147
9860 Q 709+116 T 825  ☑ 825 
9861 Q 33+675  T 708  ☑ 708 
9862 Q 47+401  T 448  ☑ 448 
9863 Q 94+712  T 806  ☑ 806 
9864 Q 70+892  T 962  ☑ 962 
9865 Q 37+80   T 117  ☑ 117 
9866 Q 328+938 T 1266 ☑ 1266
9867 Q 55+106  T 161  ☑ 161 
9868 Q 50+666  T 716  ☑ 716 
9869 Q 30+397  T 427  ☑ 427 
9870 Q 81+788  T 869  ☑ 869 
9871 Q 34+694  T 728  ☑ 728 
9872 Q 81+162  T 243  ☑ 243 
9873 Q 348+917 T 1265 ☑ 1265
9874 Q 91+860  T 951  ☑ 951 
9875 Q 306+842 T 1148 ☑ 1148
9876 Q 526+31  T 557  ☑ 557 
9877 Q 460+484 T 944  ☑ 944 
9878 Q 137+36  T 173  ☑ 173 
9879 Q 32+122  T 154  ☑ 154 
9880 Q 887+71  T 958  ☑ 958 
9881 Q 605+221 T 826  ☑ 826 
9882 Q 12+227  T 239  ☑ 239 
9883 Q 215+519 T 734  ☑ 734 
9884 Q 278+688 T 966  ☑ 966 
9885 Q 318+93  T 411  ☑ 411 
9886 Q 440+385 T 825  ☑ 825 
9887 Q 80+2    T 82   ☑ 82  
9888 Q 15+522 